reference: https://towardsdatascience.com/simple-abstractive-text-summarization-with-pretrained-t5-text-to-text-transfer-transformer-10f6d602c426

Tokenizer Info: https://huggingface.co/transformers/tokenizer_summary.html

In [1]:
from bs4 import BeautifulSoup
import urllib.request as urllib2

In [26]:
def clean_text(url, num_p):
    page = urllib2.urlopen(url)
    soup = BeautifulSoup(page, "html.parser")
    
    ps = []
    for i, paragraph in enumerate(soup.findAll("p")):
        p = paragraph.text.split()
        if len(p) == 0 or p[-1][-1] not in ".!,:;":
            continue
        ps.append(" ".join(p))
        
        if i >= num_p:
            break
   
    text = " ".join(ps)
    
    return text

In [3]:
url1 = "https://www.salesforce.com/eu/blog/2020/06/real-world-examples-of-machine-learning.html"
url2 = "https://bigdata-madesimple.com/top-10-real-life-examples-of-machine-learning/"
url3 = "https://www.forbes.com/sites/bernardmarr/2018/04/30/27-incredible-examples-of-ai-and-machine-learning-in-practice/?sh=30e992bf7502"

In [4]:
import torch
import json 
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

In [5]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained('t5-small')
device = torch.device('cpu')

In [27]:
def summarize(url, num_p):
    cleaned_text = clean_text(url, num_p)
    
    tokenized_text = tokenizer.encode(cleaned_text, return_tensors="pt").to(device)

    summary_ids = model.generate(tokenized_text,
                                        num_beams=4,
                                        no_repeat_ngram_size=2,
                                        min_length=30,
                                        max_length=100,
                                        early_stopping=True)

    output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return output

In [28]:
summarize(url1, 15)

'. Machine learning is a modern innovation that has enhanced many industrial and professional processes. Image recognition is an example of machine learning in the real world. It can identify commonalities and match them to faces. Some software applications can convert live voice and recorded speech into text. The speech can be segmented by intensities on time-frequency bands as well.'

In [29]:
summarize(url2, 15)

'. Machine learning is a subset of artificial intelligence, which focuses on using statistical techniques to build intelligent computer systems in order to learn from databases available to it. Currently, machine learning has been used in multiple fields and industries – for example, medical diagnosis, image processing, prediction, classification, learning association, regression etc.'

In [30]:
summarize(url3, 30)

', AI and machine learning can help drive high performance. BP is at the forefront of realizing the opportunities big data and artificial intelligence has for the energy industry. they use natural language processing and advanced analytics to help musicians understand what audiences want and what songs might ultimately be hits.'